# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [2]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=pronet",
        "encoder.level='aminoacid'",
        "encoder.num_blocks=4",
        "encoder.hidden_channels=128",
        "encoder.out_channels=1195",
        "encoder.mid_emb=64",
        "encoder.num_radial=6",
        "encoder.num_spherical=2",
        "encoder.cutoff=10.0",
        "encoder.max_num_neighbors=32",
        "encoder.int_emb_layers=3",
        "encoder.out_layers=2",
        "encoder.num_pos_emb=16",
        "encoder.dropout=0.3",
        "encoder.data_augment_eachlayer=True",
        "encoder.euler_noise=False",
        "encoder.pretraining=False",
        "encoder.node_embedding=False",

        "decoder.graph_label.dummy=True",

        "task=multiclass_graph_classification",
        "dataset=fold_family",
        "dataset.datamodule.batch_size=32",
        "features=ca_base", 
        "+aux_task=none",
        
        "trainer.max_epochs=1000",
        "optimiser=adam",
        "optimiser.optimizer.lr=5e-4",
        "callbacks.early_stopping.patience=20",
        "test=True",
        "scheduler=steplr",

        ## for test ONLY
        # "task_name=test",  # here
        # "ckpt_path_test=/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_pronet_fold_400epochs/checkpoints/epoch_273.ckpt", # here
        # "optimizer.weight_decay=0.5"
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/19/24 18:47:03] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=46038;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=702309;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=393888;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=173775;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=53885;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=476551;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/19/24 18:47:05] INFO     Hydra initialised at                                                    ]8;id=155200;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=162793;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [3]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=734551;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=350944;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=763933;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=22914;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=706335;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=788674;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=302310;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=210967;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [4]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'name', 'seed', 'num_workers', 'task_name', 'ckpt_path_test', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In [5]:
# import torch
# import torch.nn as nn

# # Assuming `model` is your model and `encoder_weights` is the state_dict of pretrained weights
# print(torch.load(cfg.ckpt_path).keys())
# # Load the pretrained state_dict
# checkpoint = torch.load(cfg.ckpt_path)
# pretrained_dict = torch.load(cfg.ckpt_path)["state_dict"]

# # Create a new state_dict that excludes the final layer parameters
# filtered_dict = {k: v for k, v in pretrained_dict.items() if not k.startswith('encoder.lin_out.')}
# filtered_dict = {k: v for k, v in filtered_dict.items() if not k.startswith('encoder.lins_out.')}
# print(filtered_dict.keys())
# new_checkpoint = {
#     "epoch": checkpoint["epoch"],
#     "global_step": checkpoint["global_step"],
#     "pytorch-lightning_version": checkpoint["pytorch-lightning_version"],
#     "state_dict": filtered_dict,
#     "loops": checkpoint["loops"],
#     "callbacks": checkpoint["callbacks"],
#     "optimizer_states": checkpoint["optimizer_states"],
#     "lr_schedulers": checkpoint["lr_schedulers"],
#     "hparams_name": checkpoint["hparams_name"],
#     "hyper_parameters": checkpoint["hyper_parameters"]
# }

# torch.save(new_checkpoint, '/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_pronet_pretraining_best@2/checkpoints/epoch_002_filtered.ckpt')

In [6]:
from proteinworkshop.finetune import train_model

train_model(cfg)

Seed set to 52


[07/19/24 18:47:06] INFO     Instantiating datamodule:                                              ]8;id=896664;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/finetune.py\finetune.py]8;;\:]8;id=987468;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/finetune.py#100\100]8;;\
                             <proteinworkshop.datasets.fold_classification.FoldClassificationDataMo                
                             dule...                                                                               

                    INFO     Instantiating callbacks...                                             ]8;id=166878;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/finetune.py\finetune.py]8;;\:]8;id=837528;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/finetune.py#107\107]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=1185;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=35855;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.EarlyStopping>     ]8;id=248769;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=72738;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃     ┃ Name                                           ┃ Type        ┃ Params ┃ Mode  ┃    In sizes ┃   Out sizes ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 0   │ encoder                                        │ ProNet      │  1.5 M │ train │           ? │           ? │
│ 1   │ encoder.feature0                               │ d_theta_ph… │      0 │ train │  [[181072], │    [181072, │
│     │                                                │             │        │       │   [181072], │         24] │
│     │                                                │             │        │       │   [181072]] │             │
│ 2   │ encoder.feature1                               │ d_angle_emb │      0 │ train │  [[181072], │    [181072, │
│     │                                                │             │        │       │   [181072]] │         12] │
│ 3   │ encoder.embedding                              │ Embedding   │  3.3 K │ train │     [11317] │     [11317, │
│     │                                                │             │        │       │             │        128] │
│ 4   │ encoder.interaction_blocks                     │ ModuleList  │  1.4 M │ train │           ? │           ? │
│ 5   │ encoder.interaction_blocks.0                   │ Interactio… │  340 K │ train │    [[11317, │     [11317, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │    [181072, │             │
│     │                                                │             │        │       │        24], │             │
│     │                                                │             │        │       │    [181072, │             │
│     │                                                │             │        │       │        12], │             │
│     │                                                │             │        │       │    [181072, │             │
│     │                                                │             │        │       │    16], [2, │             │
│     │                                                │             │        │       │    181072], │             │
│     │                                                │             │        │       │    [11317]] │             │
│ 6   │ encoder.interaction_blocks.0.dropout           │ Dropout     │      0 │ train │     [11317, │     [11317, │
│     │                                                │             │        │       │        128] │        128] │
│ 7   │ encoder.interaction_blocks.0.conv0             │ EdgeGraphC… │ 32.9 K │ train │    [[11317, │     [11317, │
│     │                                                │             │        │       │   128], [2, │        128] │
│     │                                                │             │        │       │    181072], │             │
│     │                                                │             │        │       │    [181072, │             │
│     │                                                │             │        │       │       128]] │             │
│ 8   │ encoder.interaction_blocks.0.conv0.aggr_module │ SumAggrega… │      0 │ train │   [[181072, │     [11317, │
│     │                                                │             │        │       │       128], │        128] │
│     │                                                │             │        │       │   [181072], │             │
│     │                                                │             │        │       │   '?', '?', │             │
│     │                                                │             │        │       │        '?'] │             │
│ 9   │ encoder.interaction_blocks.0.conv0.lin_l       │

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 6

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 7747. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3294. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



Training: |          | 0/? [00:00<?, ?it/s]

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py